In [1]:
import random
from collections import Counter, defaultdict
from pymongo import MongoClient
import pickle
import os
import json
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
from utils import *
from sklearn.metrics.pairwise import cosine_similarity
import jieba
import re
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.metrics import cohen_kappa_score
import scipy.stats as stats
from scipy.stats import pearsonr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from collections import Counter
import utils
from scipy.stats import pearsonr, spearmanr, ttest_ind, ttest_rel
from sklearn.metrics import cohen_kappa_score, f1_score
from sklearn.linear_model import LinearRegression
from itertools import combinations
import random
import os
from sklearn import metrics
from tqdm import tqdm_notebook
import re
import jieba
import json
plt.rcParams["font.family"] = "SimHei"

sns.set_context("notebook")
sns.set_style("ticks")
%matplotlib inline

In [4]:
# ----------- GENERAL OBJECTS ----------- # 
# %run "[POST-Class] System Predictions (based on mongodb).ipynb" # SysPrediction
%run "[Class] User Peer Feedback (based on gen-0).ipynb" # UserFeedback, PeerAssessment

PATH = "data/user-study/"
df_user = pd.read_pickle(os.path.join(PATH, "df_user.pkl"))
df_peer = pd.read_pickle(os.path.join(PATH, "df_peer.pkl"))
df_item = pd.read_pickle(os.path.join(PATH, "df_up_items.pkl")) # outer join

# sys_predictions = SysPrediction()
user_feedback = UserFeedback(df_user)
peer_assess = PeerAssessment(df_peer)

UPREF = "post_expected_preference"
APREF = "peer_preference"
UWATCH = "post_watch_intent"
AWATCH = "peer_watch_intent"

In [5]:
# chosen users
user_ids = [
    "u_15810795617",
    "u_17801182378",
    "u_15313346392",
    "u_18222716322",
    "u_15071347094",
    "u_18712328742",
    "u_15900292575",
    "u_19920091165",
    "u_18221771895",
    "u_18811400801",
    "u_18811195178",
    "u_19801210262",
    "u_18993873008",
    "u_13701195791",
    "u_18800182977",
    "u_18801378212"
]
df_item = df_item[df_item['user_id'].isin(user_ids)].copy()
df_user = df_user[df_user['user_id'].isin(user_ids)].copy()
df_peer = df_peer[df_peer['user_id'].isin(user_ids)].copy()

In [6]:
df_item.dropna(subset=['post_expected_preference', 'peer_preference'], inplace=True)
df_item.reset_index(drop=True, inplace=True)

In [7]:
# np.mean(df_item['self_neg_cnt']), np.mean(df_item['self_neg_cnt'] / (df_item['self_pos_cnt'] + df_item['self_normal_cnt'] + df_item['self_neg_cnt'])) 

# Attribute

In [7]:
def eval_point(peer_p, self_p):
    peer_p = set(peer_p)
    self_p = set(self_p) - set(['user'])
    
    acc = len(peer_p & self_p) / float(len(peer_p)) if len(peer_p) > 0 else np.nan
    recall = len(peer_p & self_p) / float(len(self_p)) if len(self_p) > 0 else np.nan
    jc = len(peer_p & self_p) / float(len(peer_p | self_p)) if len(peer_p | self_p) > 0 else np.nan
    f1 = 2.0 * (acc * recall) / (acc + recall) if (acc + recall) > 0 else np.nan
    return {"acc": acc, "recall": recall, "jaccard": jc, 'f1': f1}

df_item['pos_metric'] = list(map(lambda t: eval_point(t[0], t[1]), zip(df_item['peer_pos'], df_item['self_pos'])))
df_item['normal_metric'] = list(map(lambda t: eval_point(t[0], t[1]), zip(df_item['peer_normal'], df_item['self_normal'])))
df_item['neg_metric'] = list(map(lambda t: eval_point(t[0], t[1]), zip(df_item['peer_neg'], df_item['self_neg'])))

In [8]:
global_p = {"pos": 0, "neg": 0, "normal": 0}
for t in df_item['self_pos']:
    global_p['pos'] += len(t)
for t in df_item['self_normal']:
    global_p['normal'] += len(t)
for t in df_item['self_neg']:
    global_p['neg'] += len(t)
total = float(sum(global_p.values()))
global_p = dict([(tk, tv/total) for tk, tv in global_p.items()])
global_p

{'pos': 0.22758793550004178,
 'neg': 0.04052134681259922,
 'normal': 0.731890717687359}

In [9]:
random.seed(2021)

from collections import defaultdict
from tqdm import tqdm

records = defaultdict(list)
for _ in tqdm(range(100)):
    random_pos = []
    random_normal = []
    random_neg = []
    for i, tr in df_item.iterrows():
        _all = tr['self_pos'] + tr['self_normal'] + tr['self_neg']
        random.shuffle(_all)
        pos_pred = _all[:int(global_p['pos'] * len(_all))]
        normal_pred = _all[int(global_p['pos'] * len(_all)):int((global_p['normal']+global_p['pos']) * len(_all))]
        neg_pred = _all[int((global_p['normal']+global_p['pos']) * len(_all)):]
                
        random_pos.append(pos_pred)
        random_normal.append(normal_pred)
        random_neg.append(neg_pred)

    metric_pos = list(map(lambda t: eval_point(t[0], t[1]), zip(random_pos, df_item['self_pos'])))
    metric_normal = list(map(lambda t: eval_point(t[0], t[1]), zip(random_normal, df_item['self_normal'])))
    metric_neg = list(map(lambda t: eval_point(t[0], t[1]), zip(random_neg, df_item['self_neg'])))
    
    for tm in ['acc', 'recall', 'jaccard']:
        records[("pos", tm)].append(np.nanmean([t[tm] for t in metric_pos]))
        records[("normal", tm)].append(np.nanmean([t[tm] for t in metric_normal]))
        records[("neg", tm)].append(np.nanmean([t[tm] for t in metric_neg]))

random_metric = {}
for tk in records:
    random_metric[tk] = np.nanmean(records[tk])
    print (tk, np.nanmean(records[tk]))

100%|██████████| 100/100 [00:14<00:00,  6.88it/s]

('pos', 'acc') 0.20622509578544063
('normal', 'acc') 0.6777883233615232
('neg', 'acc') 0.042827586206896556
('pos', 'recall') 0.1827214052847828
('normal', 'recall') 0.7443896408953029
('neg', 'recall') 0.07807022368903556
('pos', 'jaccard') 0.09404072088886999
('normal', 'jaccard') 0.5481879719718115
('neg', 'jaccard') 0.027189974457215837


In [10]:
ds_ans = utils.PDtable()
for tk in ['pos_metric', 'normal_metric', 'neg_metric']:
    for tm in ['acc', 'recall', 'jaccard']:
        ds_ans.add(np.mean(df_item[tk].map(lambda t: t[tm])), tm)
        print (tk, tm, np.mean(df_item[tk].map(lambda t: t[tm])))
df_ans = ds_ans.to_pandas()

pos_metric acc 0.4585287776711973
pos_metric recall 0.4568296541385465
pos_metric jaccard 0.25425763540921004
normal_metric acc 0.7530864245524479
normal_metric recall 0.7455383273327524
normal_metric jaccard 0.5873674493177818
neg_metric acc 0.24837888448999568
neg_metric recall 0.3226072607260726
neg_metric jaccard 0.15220196179315676


# User History

In [11]:
client = MongoClient('mongodb://localhost:27017', username='', password="")
movie_info = client.Assess.movie_info
phase1_results = client.Assess.user_phase1_results
phase2_recommend = client.Assess.user_phase2_recommend_top3
phase2_assign = client.Assess.peer_phase1_assign
phase2_results = client.Assess.peer_phase1_results
phase2_log = client.Assess.peer_phase1_log


""" Class """
class Log:
    def __init__(self):
        # logger.debug("inited gloal log!")
        self.records = {}

    def __del__(self):
        self.save()

    def init_log(self):
        user_id = session['user_id']
        print (user_id)
        if user_id in self.records:
            print ("Duplicate user id!")
            self.records[user_id + "_dup_" + get_now_time("%Y-%m-%d")] = copy.deepcopy(self.records[user_id])
            del self.records[user_id]

        peer_list = self.init_tasks()
        if len(peer_list) <= 0:
            return False

        self.records[user_id] = {
            "user_id": user_id,
            "login_time": get_now_time(),
            "time_list": [],
            "peer_list": peer_list,
            "task_data": [],
            "task_results": [{"peer_explanations_ans": None, "peer_explanations_log": [], "peer_recommendation": None} for _ in peer_list]
        }
        self.init_task_data()
        self.user_log_file = open("log_dump/{}_{}.log".format(get_now_time("%Y-%m-%d"), user_id),'w')
        return True


    def init_tasks(self):
        user_id = session['user_id']
        assign_results = phase2_assign.find_one({"expert_id": user_id})
        if assign_results:
            return assign_results['peer_ids']
        return []

    def get_movie_attributes(self, item_id):
        movie_info = self.get_item_info(item_id)
        ans_attr = []
        namedic = {"db_directors": "导演", "db_writers": "编剧", "db_casts": "主演", "genres": "类型", "countries": "制片国家/地区"}
        for tk in ['db_directors', 'db_writers', 'db_casts']:
            tvs = movie_info[tk]
            for ti, (tv, tv_url) in enumerate(tvs):
                tans = {
                        "key": namedic[tk], 
                        "value": tv,
                        "text": """<b>{}:</b><span style="text-decoration:underline" data-toggle="tooltip" data-placement="right" data-html="true" title='<img src="{}" style="height:200px"/>'>{} </span>""".format(namedic[tk], tv_url, tv),
                        "key_last": False
                        }
                if ti == len(tvs) - 1:
                    tans['key_last'] = True
                ans_attr.append(tans)

        for tk in ['genres', 'countries']:
            tvs = movie_info[tk].split(" / ")
            for ti, tv in enumerate(tvs):
                tans = {
                        "key": namedic[tk], 
                        "value": tv,
                        "text": "<b>{}:</b>{} ".format(namedic[tk], tv),
                        "key_last": False
                        }
                if ti == len(tvs) - 1:
                    tans['key_last'] = True
                ans_attr.append(tans)

        ans_attr.append({"key": "热度（评分数量）", "value": "1", "text": "<b>热度（评分数量）</b>", "key_last": False})
        ans_attr.append({"key": "和Ta某部看过的电影相似", "value": "1", "text": """<span style="text-decoration:underline" data-toggle="tooltip" data-placement="right" data-html="true" title="<div style='width:200px'>例如：你发现这电影和Ta看过的电影A很像，但Ta不喜欢A，那可能就是负向的，反之是无影响或正向"> <b>和某部看过的电影相似</b> </span>""", "key_last": True})
        # ans_attr.append({"key": "口味相似的其它人喜欢", "value": "1", "text": "<b>口味相似的其它人喜欢</b>", "key_last": False})
        # ans_attr.append({"key": "Ta的朋友也喜欢", "value": "1", "text": "<b>Ta的朋友也喜欢</b>", "key_last": False})
        return ans_attr

    def get_item_info(self, item_id):
        if type(item_id) != int:
            item_id = int(item_id)
        tinfo = movie_info.find_one({'movieId': item_id})
        if tinfo:
            return {"title": tinfo['show_title'], 
                    "movieId": tinfo['movieId'],
                    "itemID": tinfo['show_itemID'],
                    "itemUrl": "https://movie.douban.com/subject/{}/".format(tinfo['show_itemID']), 
                    "imgUrl": "/ex2" + tinfo['show_imgUrl'],
                    "information": tinfo['show_information'],
                    "summary": tinfo['show_summary'],
                    "directors": tinfo["show_directors"],
                    "writers": tinfo["show_writers"],
                    "casts": tinfo["show_casts"],
                    "genres": tinfo["show_genres"],
                    "countries": tinfo["show_countries"],
                    "languages": tinfo["show_languages"],
                    "date": tinfo["show_date"],
                    "duration": tinfo["show_duration"],
                    "rating_count": int(tinfo['db_ratings_count']),
                    "db_directors": [(t['name'], "/ex2/static/figures/attribute/c_{}.jpg".format(t['id'])) for t in tinfo['db_directors']],
                    "db_writers": [(t['name'], "/ex2/static/figures/attribute/c_{}.jpg".format(t['id'])) for t in tinfo['db_writers']],
                    "db_casts": [(t['name'], "/ex2/static/figures/attribute/c_{}.jpg".format(t['id'])) for t in tinfo['db_casts']],
                    "aka": tinfo['show_aka']}
        return False

    def __init_single_task_data(self, peer_id, cond_amount, cond_content):
        user_id = session['user_id']

        # pre_alignment
        # phase1_results
        print (peer_id)
        user_log = phase2_recommend.find_one({"user_id": peer_id})
        user_item_list = [{"movieId": t['item_id'], "rating": int(float(t['preference'])) if t['preference'] else 0, "review": t['review']} for t in user_log['train_items']]
        # cond_amount
        random.seed(2020 + int(peer_id.split("_")[1]))
        random.shuffle(user_item_list)
        user_item_list = user_item_list[:int(cond_amount)]

        user_item_list = sorted(user_item_list, key=lambda t: t['rating'], reverse=True)

        ans_pre = []
        for titem in user_item_list:
            tinfo = self.get_item_info(titem['movieId'])
            if tinfo:
                tinfo['rating'] = titem['rating']
                tinfo['review'] = titem['review']
                # rating_review_dic = {
                #     1: "1星：很不喜欢",
                #     2: "2星：不喜欢",
                #     3: "3星：一般",
                #     4: "4星：喜欢",
                #     5: "5星：很喜欢",
                # }
                # tinfo['review'] = rating_review_dic[tinfo['rating']]
                ans_pre.append(tinfo)
            else:
                print (titem)

        # candidates
        """build by recommendation algorithm -> database"""
        user_rec = phase2_recommend.find_one({"user_id": peer_id})
        user_candidate_list = [t['item'] for t in user_rec['rec_list']]
        random.seed(2020 + int(peer_id.split("_")[1])) # TODO.
        random.shuffle(user_candidate_list)

        ans_can = []
        for titem_id in user_candidate_list:
            tinfo = self.get_item_info(titem_id)
            tinfo['attributes'] = self.get_movie_attributes(titem_id)
            if tinfo:
                ans_can.append(tinfo)
            else:
                print (tinfo)

        return {
            "peer_preferences": ans_pre,
            "peer_candidates": ans_can,
            "cond_content": cond_content,
            "cond_amount": cond_content
        }


    def init_task_data(self):
        # KEY FUNCTION
        user_id = session['user_id']
        for tidx, peer_id in enumerate(self.records[user_id]['peer_list']):
            self.records[user_id]['task_data'].append(self.__init_single_task_data(peer_id, 10, "rw"))

    def ans_user_summary(self, user_summary_ans):
        user_id = session['user_id']
        task_id = session['task_id']
        self.records[user_id]['task_results'][task_id]['user_summary_ans'] = user_summary_ans

    def add_peer_rank(self, q_ans):
        user_id = session['user_id']
        task_id = session['task_id']
        self.records[user_id]['task_results'][task_id]['peer_rank_ans'] = q_ans

    def ans_peer_explanations(self, peer_explanation_ans):
        user_id = session['user_id']
        task_id = session['task_id']
        self.records[user_id]['task_results'][task_id]['peer_explanations_ans'] = peer_explanation_ans

    def log_peer_explanation(self, single_ans):
        user_id = session['user_id']
        task_id = session['task_id']
        self.records[user_id]['task_results'][task_id]['peer_explanations_log'].append(single_ans)
        self.dump_user_log(user_id)

    def ans_peer_recommendation(self, peer_recommendation_ans):
        user_id = session['user_id']
        task_id = session['task_id']
        self.records[user_id]['task_results'][task_id]['peer_recommendation'] = peer_recommendation_ans

    def add_time(self, page):
        self.records[session['user_id']]['time_list'].append((page, get_now_time("timestamp")))

    def save(self):
        for stu_id in self.records:
            self.save_user_log(stu_id)

    def dump_user_log(self, user_id):
        phase2_log.insert_one(copy.deepcopy(self.records[user_id]))
        # logger.info("Dump log: {}".format(user_id))

    def save_user_log(self, user_id):
        # json.dump(self.records[user_id], self.user_log_file, indent=1)
        phase2_results.insert_one(copy.deepcopy(self.records[user_id]))
        # logger.info("Saved log: {}".format(user_id))

    def finish_user(self, user_id):
        # logger.info("Finish user {}".format(user_id))
        self.save_user_log(user_id)

        del self.records[user_id]
        session.clear()

    def dump_user_progress(self, user_id):
        u_progress_dump = {
            "user_id": user_id,
            "records": self.records[session['user_id']],
            "session": dict(session)
        }
        pickle.dump(u_progress_dump, open("progress_dump/{}.pkl".format(user_id), "wb"))

    def load_user_progress(self, user_id):
        if not os.path.exists("progress_dump/{}.pkl".format(user_id)):
            return False
        u_progress_dump = pickle.load(open("progress_dump/{}.pkl".format(user_id), "rb"))
        self.records[u_progress_dump['user_id']] = u_progress_dump['records']
        for tk in u_progress_dump['session']:
            session[tk] = u_progress_dump['session'][tk]
        
        print (session)
        return True
    
    def load_user_replay(self, user_id):
        if not os.path.exists("replay/{}.pkl".format(user_id)):
            return False
        u_replay = pickle.load(open("replay/{}.pkl".format(user_id), "rb"))
        self.records[user_id]['replay'] = u_replay
        print ("LOADED PREVIOUS REPLAY:", user_id)
        # print (u_replay)
        return True
        

In [12]:
log = Log()
def get_attributes(tm):
    attrs = []
    for tdic in log.get_movie_attributes(tm):
        tk = tdic['key']
        if "热度" in tk:
            attrs.append("pop")
        elif "相似" in tk:
            attrs.append("item")
        else:
            attrs.append("{}={}".format(tdic['key'], tdic['value']))
    return attrs

def parse_userlog(rec):
    user_id = rec['user_id']
    ans = []
    for tr in rec['pre_rating_results']:
        if int(tr['skip']) == 0:
            ans.append((user_id, tr['item_id'], tr['preference']))
    df_ans = pd.DataFrame.from_records(ans, columns=['user', 'item', 'rating'])
    return df_ans

In [13]:
user_ids = list(set(df_item['user_id']))
user_hist = {}
for tu in user_ids:
    ulog = parse_userlog(phase1_results.find_one({"user_id": tu}))
    ulog['attrs'] = [get_attributes(tm) for tm in ulog['item']]
    
    uattr_dic = {}
    for tattrs, tr in zip(ulog['attrs'], ulog['rating']):
        for tattr in tattrs:
            uattr_dic.setdefault(tattr, [])
            uattr_dic[tattr].append(tr)
    user_hist[tu] = uattr_dic

In [14]:
stats_pos, stats_normal, stats_neg = [], [], []

for tu, tattrs in zip(df_item['user_id'], df_item['self_attributes']):
    if type(tattrs) != list or len(tattrs) <= 0:
        stats_pos.append([])
        stats_normal.append([])
        stats_neg.append([])
        continue
        
    attrs = ["=".join(tattr.split("=")[:2]) for tattr in tattrs]
    pred_pos = []
    pred_normal = []
    pred_neg = []
    for tattr in attrs:
        if tattr in user_hist[tu]:
            if np.mean(user_hist[tu][tattr]) > 3:
                pred_pos.append(tattr)
            else:
                pred_neg.append(tattr)
        else:
            pred_normal.append(tattr)
    stats_pos.append(pred_pos)
    stats_normal.append(pred_normal)
    stats_neg.append(pred_neg)
    
df_item['stats_pos'] = stats_pos
df_item['stats_normal'] = stats_normal
df_item['stats_neg'] = stats_neg

In [15]:
df_item['stats_pos_metric'] = list(map(lambda t: eval_point(t[0], t[1]), zip(df_item['stats_pos'], df_item['self_pos'])))
df_item['stats_normal_metric'] = list(map(lambda t: eval_point(t[0], t[1]), zip(df_item['stats_normal'], df_item['self_normal'])))
df_item['stats_neg_metric'] = list(map(lambda t: eval_point(t[0], t[1]), zip(df_item['stats_neg'], df_item['self_neg'])))

In [16]:
ds_ans = utils.PDtable()
for tk in ['stats_pos_metric', 'stats_normal_metric', 'stats_neg_metric']:
    for tm in ['acc', 'recall', 'jaccard']:
        ds_ans.add(np.mean(df_item[tk].map(lambda t: t[tm])), tm)
        print (tk, tm, np.mean(df_item[tk].map(lambda t: t[tm])))
df_ans = ds_ans.to_pandas()

stats_pos_metric acc 0.3488134754919206
stats_pos_metric recall 0.3986917463711537
stats_pos_metric jaccard 0.2019224017046318
stats_normal_metric acc 0.5828884631791117
stats_normal_metric recall 0.6581381823106578
stats_normal_metric jaccard 0.445113355245636
stats_neg_metric acc 0.049999999999999996
stats_neg_metric recall 0.02915291529152916
stats_neg_metric jaccard 0.016462167689161554
